In [29]:
import pandas as pd

import datetime
from datetime import timedelta

import statistics
import math

import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
pio.templates.default = 'plotly_white'
pd.set_option("display.precision", 3)

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [34]:
buses_data_pc = pd.read_csv('../../flash/EMTBuses/ProcessedData/buses_data_pc.csv',
    dtype={
        'line': 'str',
        'destination': 'str',
        'stop': 'uint16',
        'bus': 'uint16',
        'given_coords': 'bool',
        'pos_in_burst':'uint16',
        'estimateArrive': 'int32',
        'DistanceBus': 'int32',
        'request_time': 'int32',
        'lat':'float32',
        'lon':'float32'
    }
)[['line','destination','stop','bus','datetime','estimateArrive','DistanceBus','arrival_time','given_coords','lat','lon']]


#Parse the dates
buses_data_pc['datetime'] = pd.to_datetime(buses_data_pc['datetime'], format='%Y-%m-%d %H:%M:%S.%f')
buses_data_pc['arrival_time'] = pd.to_datetime(buses_data_pc['arrival_time'], format='%Y-%m-%d %H:%M:%S.%f')

#Show the df
buses_data_pc.tail()

,line,destination,stop,bus,datetime,estimateArrive,DistanceBus,arrival_time,given_coords,lat,lon
18184046,44,MARQUES DE VIANA,1678,5453,2020-04-07 21:49:57.554833,549,2236,2020-04-07 21:59:06.554833,False,0.0,0.0
18184047,133,CALLAO,3295,5486,2020-04-07 21:49:57.554881,1067,875,2020-04-07 22:07:44.554881,False,0.0,0.0
18184048,133,CALLAO,3295,5482,2020-04-07 21:49:57.554881,167,820,2020-04-07 21:52:44.554881,False,0.0,0.0
18184049,44,MARQUES DE VIANA,1670,5453,2020-04-07 21:49:57.561538,169,644,2020-04-07 21:52:46.561538,False,0.0,0.0
18184050,44,MARQUES DE VIANA,1670,4703,2020-04-07 21:49:57.561538,1010,4523,2020-04-07 22:06:47.561538,False,0.0,0.0


In [35]:
#Coronavirus Interval
covid_start = datetime.datetime(2020,3,16)
covid_end = datetime.datetime.now()

In [152]:
#Before COVID-19 Data
buses_data_pc_2km = buses_data_pc.loc[(buses_data_pc.datetime < covid_start) & \
                                    (buses_data_pc.DistanceBus < 2000) & \
                                 (buses_data_pc.DistanceBus > 1000)].sample(10000)
buses_data_pc_1km = buses_data_pc.loc[(buses_data_pc.datetime < covid_start) & \
                                 (buses_data_pc.DistanceBus < 1000)].sample(10000)

In [153]:
def get_arrival_time_error(row):
    seconds = ((row.datetime + timedelta(seconds = row.estimateArrive)) - row.arrival_time).total_seconds()
    return seconds
    
def get_arrival_time_sq_error(row):
    return row.estim_error**2
    

buses_data_pc_2km['estim_error'] = buses_data_pc_2km.apply(get_arrival_time_error, axis=1)
#Eliminate outliers
buses_data_pc_2km = buses_data_pc_2km.loc[(buses_data_pc_2km.estim_error<300) & \
                                         (buses_data_pc_2km.estim_error>-300)].sample(5000)
buses_data_pc_2km['sq_estim_error'] = buses_data_pc_2km.apply(get_arrival_time_sq_error, axis=1)


buses_data_pc_1km['estim_error'] = buses_data_pc_1km.apply(get_arrival_time_error, axis=1)
#Eliminate outliers
buses_data_pc_1km = buses_data_pc_1km.loc[(buses_data_pc_1km.estim_error<300) & \
                                         (buses_data_pc_1km.estim_error>-300)].sample(5000)
buses_data_pc_1km['sq_estim_error'] = buses_data_pc_1km.apply(get_arrival_time_sq_error, axis=1)

In [154]:
buses_data_pc_2km.head()

,line,destination,stop,bus,datetime,estimateArrive,DistanceBus,arrival_time,given_coords,lat,lon,estim_error,sq_estim_error
6496770,1,CRISTO REY,1750,119,2020-03-11 10:22:38.245102,664,1312,2020-03-11 10:33:39.001620,True,40.430,-3.670,3.243,10.520
738052,132,MONCLOA,1367,4828,2020-02-27 18:05:32.333634,463,1864,2020-02-27 18:14:22.715810,True,40.483,-3.702,-67.382,4540.358
182351,1,CRISTO REY,1750,116,2020-02-26 19:24:57.141817,662,1169,2020-02-26 19:35:22.043208,True,40.430,-3.674,37.099,1376.307
1699000,132,HOSPITAL LA PAZ,1366,4827,2020-02-29 22:25:29.557214,419,1878,2020-02-29 22:32:05.115825,True,40.463,-3.713,23.441,549.499
196340,132,MONCLOA,1331,4739,2020-02-26 19:38:56.343818,556,1972,2020-02-26 19:47:56.232555,True,40.481,-3.704,16.111,259.573


In [155]:
buses_data_pc_1km.head()

,line,destination,stop,bus,datetime,estimateArrive,DistanceBus,arrival_time,given_coords,lat,lon,estim_error,sq_estim_error
3211615,82,MONCLOA,1630,4715,2020-03-04 08:32:41.441988,62,445,2020-03-04 08:34:15.578721,True,40.464,-3.724,-32.137,1032.770
3796437,F,CUATRO CAMINOS,4291,8607,2020-03-05 10:40:10.777561,202,483,2020-03-05 10:43:37.018462,True,40.446,-3.723,-4.241,17.985
2678999,82,PITIS,1711,4822,2020-03-03 07:48:43.417306,148,809,2020-03-03 07:51:46.493448,True,40.461,-3.727,-35.076,1230.336
5115632,82,MONCLOA,4022,4703,2020-03-08 11:47:52.250758,146,578,2020-03-08 11:49:42.075318,False,40.438,-3.719,36.175,1308.662
3011740,1,PROSPERIDAD,4514,121,2020-03-03 18:01:58.723849,61,0,2020-03-03 18:02:59.543002,True,40.438,-3.718,0.181,0.033


In [160]:
#Before COVID-19 Histogram of Estimation Errors
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=buses_data_pc_1km.estim_error,name='Error when the distance to the stop is between 0 and 1km',
    histnorm='probability',
    xbins=dict(
        size=5
    )
))
fig.add_trace(go.Histogram(
    x=buses_data_pc_2km.estim_error,name='Error when the distance to the stop is between 1 and 2km',
    histnorm='probability',
    xbins=dict(
        size=5
    )
))
# Overlay both histograms
fig.update_layout(
    title='Error between API estimations and estimated real arrival times depending on the distance to the stop',
    xaxis_title='Seconds',
    yaxis_title='Normalized Count',
    showlegend=False
)
fig.show()

In [161]:
def compute_results(df) :
    N = df.shape[0] 
    rmse = math.sqrt(df.sq_estim_error.sum()/N)
    mean_error = df.estim_error.sum()/N
    
    def estim_deviation(row) :
        return (mean_error-row.estim_error)**2
    
    df['estim_dev'] = df.apply(estim_deviation, axis=1)
    
    std_error = math.sqrt(df.estim_dev.sum()/N-1)
    
    return rmse,mean_error,std_error

In [158]:
rmse,mean_error,std_error = compute_results(buses_data_pc_1km)
rmse,mean_error,std_error

(51.78599249071874, -4.6161135826, 51.57015138276514)

In [159]:
rmse,mean_error,std_error = compute_results(buses_data_pc_2km)
rmse,mean_error,std_error

(81.48499271196839, -12.4814244178, 80.51719121884648)